In [ ]:
# https://pytorch.org/ cuda 설치, pytorch 설치 
# !conda install pytorch torchvision torchaudio pytorch-cuda=11.6 -c pytorch -c nvidia

In [ ]:
!pip install sklearn

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 모듈 import 및 전역 변수 설정

In [ ]:
# 코랩에서 실행 시 drive 마운트
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import os

import torch
import torch.nn as nn
from tqdm import trange

from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader, TensorDataset
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW

from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
import copy

import random

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import pickle

for_test = True

PADDING_TOKEN = 1
S_OPEN_TOKEN = 0
S_CLOSE_TOKEN = 2

do_eval=True
FULL_FINETUNING = True
category_extraction_model_path = '/content/drive/MyDrive/게임/korean_ABSA_baseline-main/saved_model5/category_extraction'
polarity_classification_model_path = '/content/drive/MyDrive/게임/korean_ABSA_baseline-main/saved_model5/polarity_classification'


train_data_path = '/content/drive/MyDrive/게임/korean_ABSA_baseline-main/data/nikluge-sa-2022-train.jsonl'
dev_data_path = '/content/drive/MyDrive/게임/korean_ABSA_baseline-main/data/nikluge-sa-2022-dev.jsonl'
test_data_path = '/content/drive/MyDrive/게임/korean_ABSA_baseline-main/data/nikluge-sa-2022-test.jsonl'



max_len = 256
batch_size = 8
# base_model = 'xlm-roberta-base'
base_model = 'klue/roberta-large'
entity_property_learning_rate = 3e-6
polarity_learning_rate = 3e-6
eps = 1e-8
num_train_epochs = 100
classifier_hidden_size = 768
classifier_dropout_prob = 0.1

entity_property_patience_ub = 10
polarity_patience_ub = 10
val_num = 5

entity_property_pair = ['본품#품질',
'제품 전체#일반',
'제품 전체#품질',
'본품#일반',
'제품 전체#디자인',
'본품#편의성',
'제품 전체#편의성',
'제품 전체#인지도',
'패키지/구성품#디자인',
'브랜드#일반',
'제품 전체#가격',
'패키지/구성품#편의성',
'패키지/구성품#일반',
# '본품#다양성',
# '본품#디자인',
# '브랜드#품질',
# '패키지/구성품#품질',
# '브랜드#인지도',
# '브랜드#가격',
# '패키지/구성품#다양성',
# '본품#가격',
# '본품#인지도',
# '패키지/구성품#가격'
]

tf_id_to_name = ['True', 'False']
tf_id_to_name = ['본품#품질',
'제품 전체#일반',
'제품 전체#품질',
'본품#일반',
'제품 전체#디자인',
'본품#편의성',
'제품 전체#편의성',
'제품 전체#인지도',
'패키지/구성품#디자인',
'브랜드#일반',
'제품 전체#가격',
'패키지/구성품#편의성',
'패키지/구성품#일반',
# '본품#다양성',
# '본품#디자인',
# '브랜드#품질',
# '패키지/구성품#품질',
# '브랜드#인지도',
# '브랜드#가격',
# '패키지/구성품#다양성',
# '본품#가격',
# '본품#인지도',
# '패키지/구성품#가격'
]
tf_name_to_id = {tf_id_to_name[i]: i for i in range(len(tf_id_to_name))}

polarity_id_to_name = ['positive', 'neutral', 'negative']
polarity_name_to_id = {polarity_id_to_name[i]: i for i in range(len(polarity_id_to_name))}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

special_tokens_dict = {
    'additional_special_tokens': ['&name&', '&affiliation&', '&social-security-num&', '&tel-num&', '&card-num&', '&bank-account&', '&num&', '&online-account&']
}

json 및 jsonl 파일 read, write 함수

In [ ]:
def jsonload(fname, encoding="utf-8"):
    with open(fname, encoding=encoding) as f:
        j = json.load(f)

    return j


# json 개체를 파일이름으로 깔끔하게 저장
def jsondump(j, fname):
    with open(fname, "w", encoding="UTF8") as f:
        json.dump(j, f, ensure_ascii=False)

# jsonl 파일 읽어서 list에 저장
def jsonlload(fname, encoding="utf-8"):
    json_list = []
    with open(fname, encoding=encoding) as f:
        for line in f.readlines():
            json_list.append(json.loads(line))
    return json_list


# jsonlload('/content/drive/MyDrive/게임/korean_ABSA_baseline-main/data/sample.jsonl')

[{'id': 'nikluge-sa-2022-train-00001',
  'sentence_form': '둘쨋날은 미친듯이 밟아봤더니 기어가 헛돌면서 틱틱 소리가 나서 경악.',
  'annotation': [['본품#품질', ['기어', 16, 18], 'negative']]},
 {'id': 'nikluge-sa-2022-train-00002',
  'sentence_form': '이거 뭐 삐꾸를 준 거 아냐 불안하고, 거금 투자한 게 왜 이래.. 싶어서 정이 확 떨어졌는데 산 곳 가져가서 확인하니 기어 텐션 문제라고 고장 아니래.',
  'annotation': [['본품#품질', ['기어 텐션', 67, 72], 'negative']]},
 {'id': 'nikluge-sa-2022-train-00003',
  'sentence_form': '간사하게도 그 이후에는 라이딩이 아주 즐거워져서 만족스럽게 탔다.',
  'annotation': [['제품 전체#일반', [None, 0, 0], 'positive']]},
 {'id': 'nikluge-sa-2022-train-00004',
  'sentence_form': '샥이 없는 모델이라 일반 도로에서 타면 노면의 진동 때문에 손목이 덜덜덜 떨리고 이가 부딪칠 지경인데 이마저도 며칠 타면서 익숙해지니 신경쓰이지 않게 됐다.',
  'annotation': [['제품 전체#일반', ['샥이 없는 모델', 0, 8], 'neutral']]},
 {'id': 'nikluge-sa-2022-train-00005',
  'sentence_form': '안장도 딱딱해서 엉덩이가 아팠는데 무시하고 타고 있다.',
  'annotation': [['본품#일반', ['안장', 0, 2], 'negative']]},
 {'id': 'nikluge-sa-2022-train-00006',
  'sentence_form': '지금 내 실력과 저질 체력으로는 이 정도 자전거도 되게 훌륭한 거라는..',
  'annotation'

In [ ]:
tmp_list = jsonlload(train_data_path)
tmp_list.extend(jsonlload(dev_data_path))
tmp_df = pd.DataFrame()
k=0
for i in range(len(tmp_list)):
  for j in range(len(tmp_list[i]['annotation'])):
    tmp_df.loc[k,'entity_property'] = tmp_list[i]['annotation'][j][0]
    tmp_df.loc[k,'polarity'] = tmp_list[i]['annotation'][j][-1]
    k+=1
tmp_df

,entity_property,polarity
0,본품#품질,negative
1,본품#품질,negative
2,제품 전체#일반,positive
3,제품 전체#일반,neutral
4,본품#일반,negative
...,...,...
6329,패키지/구성품#일반,positive
6330,제품 전체#가격,negative
6331,제품 전체#가격,negative
6332,제품 전체#일반,positive


In [ ]:
tmp_df['entity_property'].value_counts()

본품#품질          2380
제품 전체#일반       1624
제품 전체#품질        493
본품#일반           491
제품 전체#디자인       286
본품#편의성          191
제품 전체#편의성       180
제품 전체#인지도       141
패키지/구성품#디자인     117
브랜드#일반          103
제품 전체#가격         92
패키지/구성품#편의성      65
패키지/구성품#일반       50
본품#다양성           31
본품#디자인           21
브랜드#품질           19
패키지/구성품#품질       19
브랜드#인지도          17
브랜드#가격            7
패키지/구성품#다양성       3
본품#가격             2
본품#인지도            1
패키지/구성품#가격        1
Name: entity_property, dtype: int64

In [ ]:
import numpy as np
class_weight = list(len(tmp_df['entity_property'])/(tmp_df['entity_property'].value_counts()+100)) # entity_property의 각각 label의 positive에 대해 줄 가중치샘플비율대로 하면 너무 심해서 로그씌움.
class_weight

[2.5540322580645163,
 3.6740139211136893,
 10.681281618887015,
 10.717428087986464,
 16.409326424870468,
 21.766323024054984,
 22.62142857142857,
 26.282157676348547,
 29.1889400921659,
 31.201970443349754,
 32.989583333333336,
 38.38787878787879,
 42.22666666666667,
 48.35114503816794,
 52.34710743801653,
 53.226890756302524,
 53.226890756302524,
 54.136752136752136,
 59.19626168224299,
 61.49514563106796,
 62.09803921568628,
 62.71287128712871,
 62.71287128712871]

In [ ]:
import numpy as np
class_weight = list(len(tmp_df['entity_property'])/(tmp_df['entity_property'].value_counts()+100)) # entity_property의 각각 label의 positive에 대해 줄 가중치샘플비율대로 하면 너무 심해서 로그씌움.
class_weight = class_weight[:-10]
class_weight

[2.5540322580645163,
 3.6740139211136893,
 10.681281618887015,
 10.717428087986464,
 16.409326424870468,
 21.766323024054984,
 22.62142857142857,
 26.282157676348547,
 29.1889400921659,
 31.201970443349754,
 32.989583333333336,
 38.38787878787879,
 42.22666666666667]

In [ ]:
polarity_class_weight = list(len(tmp_df['polarity'])/tmp_df['polarity'].value_counts())
polarity_class_weight

[1.0385309067060173, 42.51006711409396, 73.65116279069767]

In [ ]:
polarity_class_weight = list(np.sqrt(len(tmp_df['polarity'])/(tmp_df['polarity'].value_counts())))
polarity_class_weight

[1.0190833659254857, 6.5199744718897445, 8.582025564556288]

# 모델 정의
roberta large 모델을 기반으로 한 classification 모델 이용

In [ ]:
class SimpleClassifier(nn.Module):

    def __init__(self, num_label):
        super().__init__()
        # self.dense = nn.Linear(classifier_hidden_size, classifier_hidden_size)
        self.dense = nn.Linear(1024, 50)
        self.dropout = nn.Dropout(classifier_dropout_prob)
        self.output = nn.Linear(50, num_label)

    def forward(self, features):
        x = features[:, 0, :]
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.output(x)
        return x


class RoBertaBaseClassifier(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(RoBertaBaseClassifier, self).__init__()

        self.num_label = num_label
        self.xlm_roberta = AutoModel.from_pretrained(base_model)
        for param in self.xlm_roberta.parameters():
            param.requires_grad = False

        self.xlm_roberta.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.xlm_roberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        # logits = torch.sigmoid(logits)

        loss = None

        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss(
                pos_weight=torch.tensor(class_weight).to(device)
                )
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels)
        # logits = torch.sigmoid(logits)

        return loss, logits

class RoBertaBaseClassifier_polar(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(RoBertaBaseClassifier_polar, self).__init__()

        self.num_label = num_label
        self.xlm_roberta = AutoModel.from_pretrained(base_model)
        for param in self.xlm_roberta.parameters():
            param.requires_grad = False

        self.xlm_roberta.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.xlm_roberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(
                weight=torch.tensor(polarity_class_weight).to(device)
                )
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits


# 데이터 파싱 및 tokenization 함수 정의


In [ ]:

def tokenize_and_align_labels(tokenizer, form, annotations, max_len):

    entity_property_data_dict = {
        'input_ids': [],
        'attention_mask': [],
        'label': []
    }
    polarity_data_dict = {
        'input_ids': [],
        'attention_mask': [],
        'label': []
    }
    tokenized_data = tokenizer(form, '미정', padding='max_length', max_length=max_len, truncation=True)
    entity_property_data_dict['input_ids'].append(tokenized_data['input_ids'])
    entity_property_data_dict['attention_mask'].append(tokenized_data['attention_mask'])
    entity_property_data_dict['label'].append([0.0]*13)
    isPairInOpinion = False
    for pair in entity_property_pair:
        
        if pd.isna(form):
            break
        

        for annotation in annotations:
            entity_property = annotation[0]
            polarity = annotation[2]

            if polarity == '------------':
                continue

            if entity_property == pair:
                entity_property_data_dict['label'][-1][tf_name_to_id[entity_property]] = 1.0

                tokenized_data = tokenizer(form, pair, padding='max_length', max_length=max_len, truncation=True)
                polarity_data_dict['input_ids'].append(tokenized_data['input_ids'])
                polarity_data_dict['attention_mask'].append(tokenized_data['attention_mask'])
                polarity_data_dict['label'].append(polarity_name_to_id[polarity])

                break


    return entity_property_data_dict, polarity_data_dict


def get_dataset(raw_data, tokenizer, max_len):
    input_ids_list = []
    attention_mask_list = []
    token_labels_list = []

    polarity_input_ids_list = []
    polarity_attention_mask_list = []
    polarity_token_labels_list = []

    for utterance in raw_data:
        entity_property_data_dict, polarity_data_dict = tokenize_and_align_labels(tokenizer, utterance['sentence_form'], utterance['annotation'], max_len)
        input_ids_list.extend(entity_property_data_dict['input_ids'])
        attention_mask_list.extend(entity_property_data_dict['attention_mask'])
        token_labels_list.extend(entity_property_data_dict['label'])

        polarity_input_ids_list.extend(polarity_data_dict['input_ids'])
        polarity_attention_mask_list.extend(polarity_data_dict['attention_mask'])
        polarity_token_labels_list.extend(polarity_data_dict['label'])

    return TensorDataset(torch.tensor(input_ids_list), torch.tensor(attention_mask_list),
                         torch.tensor(token_labels_list)), TensorDataset(torch.tensor(polarity_input_ids_list), torch.tensor(polarity_attention_mask_list),
                         torch.tensor(polarity_token_labels_list))



# 모델 학습

In [ ]:
# evaluation에 5-fold 정도로 dev 나눠서 시행하고, 성능 감소하면 full 학습으로 변경, 2연속 감소하면 중단. 5개 평균으로 예측하게 변경하기

In [ ]:

def train_sentiment_analysis():

    print('train_sentiment_analysis')
    print('category_extraction model would be saved at ', category_extraction_model_path)
    print('polarity model would be saved at ', polarity_classification_model_path)

    print('loading train data')

    train_data = jsonlload(train_data_path)
    train_data.extend(jsonlload(dev_data_path))
    tmp_df = pd.DataFrame()
    tmp_train = []
    k=0
    for i in range(len(train_data)):
      if train_data[i]['annotation'][0][0] in entity_property_pair:
        tmp_train.append(train_data[i])
        tmp_df.loc[k,'entity_property'] = train_data[i]['annotation'][0][0]
        k += 1
    from sklearn.model_selection import StratifiedKFold
    skf = StratifiedKFold(n_splits=5, random_state = 0, shuffle=True)
    val_step = 0
    for train_index, test_index in skf.split(tmp_train, tmp_df['entity_property']):
      print('\n\n======================= {} step=========================\n\n\n\n\n\n\n\n'.format(val_step+1))
      tmp_dev_data = []
      tmp_train_data = []
      for i in test_index:
        tmp_dev_data.append(tmp_train[i])
      for i in train_index:
        tmp_train_data.append(tmp_train[i])


      entity_property_val_best_loss = 100

      print('tokenizing train data')
      tokenizer = AutoTokenizer.from_pretrained(base_model)
      num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
      print('We have added', num_added_toks, 'tokens')
      entity_property_train_data, polarity_train_data = get_dataset(tmp_train_data, tokenizer, max_len)
      entity_property_dev_data, polarity_dev_data = get_dataset(tmp_dev_data, tokenizer, max_len)
      entity_property_train_dataloader = DataLoader(entity_property_train_data, shuffle=True,
                                    batch_size=batch_size)
      entity_property_dev_dataloader = DataLoader(entity_property_dev_data, shuffle=True,
                                  batch_size=batch_size)
      
      print('loading model')
      entity_property_model = RoBertaBaseClassifier(len(tf_id_to_name), len(tokenizer))
      entity_property_model.to(device)

      print('end loading')

      # entity_property_model_optimizer_setting
      FULL_FINETUNING = True
      if FULL_FINETUNING:
          entity_property_param_optimizer = list(entity_property_model.named_parameters())
          no_decay = ['bias', 'gamma', 'beta']
          entity_property_optimizer_grouped_parameters = [
              {'params': [p for n, p in entity_property_param_optimizer if not any(nd in n for nd in no_decay)],
              'weight_decay_rate': 0.01},
              {'params': [p for n, p in entity_property_param_optimizer if any(nd in n for nd in no_decay)],
              'weight_decay_rate': 0.0}
          ]
      else:
          entity_property_param_optimizer = list(entity_property_model.classifier.named_parameters())
          entity_property_optimizer_grouped_parameters = [{"params": [p for n, p in entity_property_param_optimizer]}]

      entity_property_optimizer = AdamW(
          entity_property_optimizer_grouped_parameters,
          lr=entity_property_learning_rate,
          eps=eps
      )
      epochs = num_train_epochs
      max_grad_norm = 1.0
      total_steps = epochs * len(entity_property_train_dataloader)

      entity_property_scheduler = get_linear_schedule_with_warmup(
          entity_property_optimizer,
          num_warmup_steps=0,
          num_training_steps=total_steps
      )

      epoch_step = 0
      entity_property_patience = 0

      for _ in trange(epochs, desc="Epoch"):
          
          entity_property_model.train()
          epoch_step += 1
          if entity_property_patience == int(entity_property_patience_ub*3/5):
              model_load_path = category_extraction_model_path + 'saved_model_epoch_' + str(val_step) + '.pt'
              entity_property_model.load_state_dict(torch.load(model_load_path, map_location=device))
              entity_property_model.to(device)
              for param in entity_property_model.xlm_roberta.parameters():
                  param.requires_grad = True
              entity_property_model.train()

          # entity_property train
          entity_property_total_loss = 0
          if entity_property_patience <= entity_property_patience_ub:
              for step, batch in enumerate(entity_property_train_dataloader):
                  batch = tuple(t.to(device) for t in batch)
                  b_input_ids, b_input_mask, b_labels = batch

                  entity_property_model.zero_grad()
                  # print('b_labels: ', b_labels)
                  loss, _ = entity_property_model(b_input_ids, b_input_mask, b_labels)

                  loss.backward()

                  entity_property_total_loss += loss.item()
                  # print('batch_loss: ', loss.item())

                  torch.nn.utils.clip_grad_norm_(parameters=entity_property_model.parameters(), max_norm=max_grad_norm)
                  entity_property_optimizer.step()
                  entity_property_scheduler.step()

              avg_train_loss = entity_property_total_loss / len(entity_property_train_dataloader)
              print("Entity_Property_Epoch: ", epoch_step)
              print("Average train loss: {}".format(avg_train_loss))



              if do_eval:
                  entity_property_model.eval()

                  pred_list = []
                  label_list = []
                  loss_list = []
                  for batch in entity_property_dev_dataloader:
                      batch = tuple(t.to(device) for t in batch)
                      b_input_ids, b_input_mask, b_labels = batch

                      with torch.no_grad():
                          loss, logits = entity_property_model(b_input_ids, b_input_mask, b_labels)
                          loss_list.append(loss.item())
                  print('validation error: ', np.mean(loss_list))
                  if np.mean(loss_list) < entity_property_val_best_loss:

                      entity_property_val_best_loss = np.mean(loss_list)
                      model_saved_path = category_extraction_model_path + 'saved_model_epoch_' + str(val_step) + '.pt'
                      torch.save(entity_property_model.state_dict(), model_saved_path)
                      print('entity_property_model best model updated.')
                      entity_property_patience = 0
                  else:
                      entity_property_patience += 1

      val_step += 1


    train_data = jsonlload(train_data_path)
    train_data.extend(jsonlload(dev_data_path))

    tmp_df = pd.DataFrame()
    for i in range(len(train_data)):
      tmp_df.loc[i,'polarity'] = train_data[i]['annotation'][0][-1]

    skf = StratifiedKFold(n_splits=5, random_state = 0, shuffle=True)
    val_step = 0
    for train_index, test_index in skf.split(train_data, tmp_df['polarity']):
      tmp_dev_data = []
      tmp_train_data = []
      for i in test_index:
        tmp_dev_data.append(train_data[i])
      for i in train_index:
        tmp_train_data.append(train_data[i])

      polarity_val_best_loss = 100
      print('\n\n======================= {} step=========================\n\n\n\n\n\n\n\n'.format(val_step+1))

      print('tokenizing train data')
      tokenizer = AutoTokenizer.from_pretrained(base_model)
      num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
      print('We have added', num_added_toks, 'tokens')

      entity_property_train_data, polarity_train_data = get_dataset(tmp_train_data, tokenizer, max_len)
      entity_property_dev_data, polarity_dev_data = get_dataset(tmp_dev_data, tokenizer, max_len)
      
      polarity_train_dataloader = DataLoader(polarity_train_data, shuffle=True,
                                                    batch_size=batch_size)
      polarity_dev_dataloader = DataLoader(polarity_dev_data, shuffle=True,
                                                  batch_size=batch_size)

      print('loading model')
      polarity_model = RoBertaBaseClassifier_polar(len(polarity_id_to_name), len(tokenizer))
      polarity_model.to(device)


      print('end loading')
      # polarity_model_optimizer_setting
      if FULL_FINETUNING:
          polarity_param_optimizer = list(polarity_model.named_parameters())
          no_decay = ['bias', 'gamma', 'beta']
          polarity_optimizer_grouped_parameters = [
              {'params': [p for n, p in polarity_param_optimizer if not any(nd in n for nd in no_decay)],
              'weight_decay_rate': 0.01},
              {'params': [p for n, p in polarity_param_optimizer if any(nd in n for nd in no_decay)],
              'weight_decay_rate': 0.0}
          ]
      else:
          polarity_param_optimizer = list(polarity_model.classifier.named_parameters())
          polarity_optimizer_grouped_parameters = [{"params": [p for n, p in polarity_param_optimizer]}]

      polarity_optimizer = AdamW(
          polarity_optimizer_grouped_parameters,
          lr=polarity_learning_rate,
          eps=eps
      )
      epochs = num_train_epochs
      max_grad_norm = 1.0
      total_steps = epochs * len(polarity_train_dataloader)

      polarity_scheduler = get_linear_schedule_with_warmup(
          polarity_optimizer,
          num_warmup_steps=0,
          num_training_steps=total_steps
      )


      epoch_step = 0
      polarity_patience = 0
      for _ in trange(epochs, desc="Epoch"):
          polarity_model.train()
          epoch_step += 1
          if polarity_patience == int(polarity_patience_ub*3/5):
              model_load_path = polarity_classification_model_path + 'saved_model_epoch_' + str(val_step) + '.pt'
              polarity_model.load_state_dict(torch.load(model_load_path, map_location=device))
              polarity_model.to(device)
              for param in polarity_model.xlm_roberta.parameters():
                  param.requires_grad = True
              polarity_model.train()
          if polarity_patience <= polarity_patience_ub:
              polarity_total_loss = 0

              for step, batch in enumerate(polarity_train_dataloader):
                  batch = tuple(t.to(device) for t in batch)
                  b_input_ids, b_input_mask, b_labels = batch

                  polarity_model.zero_grad()

                  loss, _ = polarity_model(b_input_ids, b_input_mask, b_labels)

                  loss.backward()

                  polarity_total_loss += loss.item()
                  # print('batch_loss: ', loss.item())

                  torch.nn.utils.clip_grad_norm_(parameters=polarity_model.parameters(), max_norm=max_grad_norm)
                  polarity_optimizer.step()
                  polarity_scheduler.step()

              avg_train_loss = polarity_total_loss / len(polarity_train_dataloader)
              print("Polarity_Epoch: ", epoch_step)
              print("Average train loss: {}".format(avg_train_loss))


              if do_eval:
                  polarity_model.eval()

                  pred_list = []
                  label_list = []
                  loss_list = []
                  for batch in polarity_dev_dataloader:
                      batch = tuple(t.to(device) for t in batch)
                      b_input_ids, b_input_mask, b_labels = batch

                      with torch.no_grad():
                          loss, logits = polarity_model(b_input_ids, b_input_mask, b_labels)
                          loss_list.append(loss.item())
                  print('validation error: ', np.mean(loss_list))
                  if np.mean(loss_list) < polarity_val_best_loss:
                      polarity_val_best_loss = np.mean(loss_list)
                      model_saved_path = polarity_classification_model_path + 'saved_model_epoch_' + str(val_step) + '.pt'
                      torch.save(polarity_model.state_dict(), model_saved_path)
                      print('\npolarity_model best model updated.')
                      polarity_patience = 0
                  else:
                      polarity_patience += 1



      print("training is done")


      val_step += 1





In [ ]:
train_sentiment_analysis()

train_sentiment_analysis
category_extraction model would be saved at  /content/drive/MyDrive/게임/korean_ABSA_baseline-main/saved_model5/category_extraction
polarity model would be saved at  /content/drive/MyDrive/게임/korean_ABSA_baseline-main/saved_model5/polarity_classification
loading train data


======================= 1 step=========================








tokenizing train data
We have added 8 tokens
loading model


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

end loading


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

Entity_Property_Epoch:  1
Average train loss: 1.1323650358850381
validation error:  1.1188477569526725


...

validation error:  0.41004821943045644
Polarity_Epoch:  23
Average train loss: 0.09492749050840711


Epoch:  23%|██▎       | 23/100 [1:09:23<4:22:20, 204.42s/it]

validation error:  0.4113460367447452
Polarity_Epoch:  24
Average train loss: 0.07238054179933613


Epoch: 100%|██████████| 100/100 [1:12:48<00:00, 43.68s/it]

validation error:  0.3929034569361982
training is done
